# Pytorch tutorial for image classification
Checkout the Pytorch tutorial in this [link](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [60]:
import torch
import torchvision
import torchvision.transforms as transforms

## Loading the Data (CIFAR10)
Downloading the data, normalize it and transform to tensors, and wrap it in DataLoaders for 'train' and 'test'.

In [61]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [62]:
from pprint import pprint
dataset_statistic = {classname: 0 for classname in classes}
for data in trainset:
    inputs, labels = data
    dataset_statistic[classes[labels]] += 1

pprint(dataset_statistic)

{'bird': 5000,
 'car': 5000,
 'cat': 5000,
 'deer': 5000,
 'dog': 5000,
 'frog': 5000,
 'horse': 5000,
 'plane': 5000,
 'ship': 5000,
 'truck': 5000}


## The Model
Define a neural network model.
The model is defined as a class which inherits from the pytorch class `Model`.
Notice that first, we define the layers in the `__init__` and only after we use the defined layers in the `forward`.
This writing is similar for tensorflow as well.

In [63]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

We define the optimizer and the loss function, which defines how to train our model:

In [64]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training
The training process:

In [65]:
from ToGrowABackbone.training_utils import evaluate

for epoch in range(6):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch}:')
    evaluate(net, testloader, classes)

print('Finished Training')

Epoch 0:
Accuracy for class plane is: 62.7 %
Accuracy for class car   is: 62.0 %
Accuracy for class bird  is: 25.7 %
Accuracy for class cat   is: 28.5 %
Accuracy for class deer  is: 38.0 %
Accuracy for class dog   is: 49.5 %
Accuracy for class frog  is: 66.9 %
Accuracy for class horse is: 58.3 %
Accuracy for class ship  is: 42.6 %
Accuracy for class truck is: 61.3 %
Epoch 1:
Accuracy for class plane is: 72.3 %
Accuracy for class car   is: 71.3 %
Accuracy for class bird  is: 29.9 %
Accuracy for class cat   is: 63.1 %
Accuracy for class deer  is: 57.9 %
Accuracy for class dog   is: 22.4 %
Accuracy for class frog  is: 56.0 %
Accuracy for class horse is: 61.3 %
Accuracy for class ship  is: 62.2 %
Accuracy for class truck is: 61.5 %
Epoch 2:
Accuracy for class plane is: 71.7 %
Accuracy for class car   is: 75.5 %
Accuracy for class bird  is: 23.7 %
Accuracy for class cat   is: 43.6 %
Accuracy for class deer  is: 64.6 %
Accuracy for class dog   is: 35.2 %
Accuracy for class frog  is: 68.5 %
A

# Evaluation
Evaluate our model on the test set.

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))